<a href="https://colab.research.google.com/github/lesteraiof/TESISMDW/blob/main/NDW_LOGS_REALTIME_FILES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
"""
Sistema de Detección de Fallas en Tiempo Real para WebLogic MDW
Versión final - Solo datos reales de archivos, sin simulación
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import warnings
import os
warnings.filterwarnings('ignore')

# Librerías para ML
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
import pickle

# Para procesamiento
import re

# ============================================================================
# 1. PARSER DE ARCHIVOS ESPECÍFICO PARA WEBLOGIC
# ============================================================================

class WebLogicLogParser:
    """Parser específico para archivos WebLogic"""

    def __init__(self):
        self.formats = {
            'weblogic': self._parse_weblogic_line,
            'apache': self._parse_apache_line,
            'generic': self._parse_generic_line
        }

    def parse_file(self, filepath, max_lines=10000):
        """Parsear archivo y detectar formato automáticamente"""
        filename = os.path.basename(filepath)
        file_size = os.path.getsize(filepath)

        print(f"Leyendo archivo: {filename}")
        print(f"  Tamano: {file_size:,} bytes")

        # Detectar formato basado en el nombre y contenido
        format_type = self._detect_format(filepath)
        print(f"  Formato detectado: {format_type}")

        logs = []
        line_count = 0

        try:
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                for line in f:
                    if line_count >= max_lines:
                        break

                    parsed_log = self.formats[format_type](line.strip(), line_count)
                    if parsed_log:
                        logs.append(parsed_log)

                    line_count += 1

                    # Mostrar progreso cada 1000 líneas
                    if line_count % 1000 == 0:
                        print(f"  Procesadas {line_count} líneas...")

            print(f"  Total de lineas: {line_count}")
            print(f"  Logs parseados: {len(logs)}")

        except Exception as e:
            print(f"  Error al leer archivo: {e}")

        return logs, line_count

    def _detect_format(self, filepath):
        """Detectar formato del archivo"""
        filename = os.path.basename(filepath).lower()

        # Verificar primeras líneas para determinar formato
        try:
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                first_lines = [f.readline() for _ in range(5)]

            # Detectar formato WebLogic
            if any('####<' in line for line in first_lines):
                return 'weblogic'
            # Detectar formato Apache/access log
            elif any('192.168' in line and 'POST' in line for line in first_lines):
                return 'apache'
            # Formato genérico por defecto
            else:
                return 'generic'

        except:
            return 'generic'

    def _parse_weblogic_line(self, line, line_num):
        """Parsear línea de log WebLogic"""
        if not line or len(line) < 20:
            return None

        log_entry = {
            'timestamp': datetime.now() - timedelta(seconds=line_num),
            'line_number': line_num,
            'original_line': line[:200],
            'source': 'weblogic',
            'log_type': 'jdbc'
        }

        # Extraer métricas de WebLogic JDBC
        metrics = self._extract_weblogic_metrics(line)
        log_entry.update(metrics)

        return log_entry

    def _parse_apache_line(self, line, line_num):
        """Parsear línea de access log"""
        if not line or len(line) < 20:
            return None

        log_entry = {
            'timestamp': datetime.now() - timedelta(seconds=line_num),
            'line_number': line_num,
            'original_line': line[:200],
            'source': 'apache',
            'log_type': 'access'
        }

        # Extraer métricas de access log
        metrics = self._extract_apache_metrics(line)
        log_entry.update(metrics)

        return log_entry

    def _parse_generic_line(self, line, line_num):
        """Parsear línea genérica"""
        if not line or len(line) < 20:
            return None

        log_entry = {
            'timestamp': datetime.now() - timedelta(seconds=line_num),
            'line_number': line_num,
            'original_line': line[:200],
            'source': 'generic',
            'log_type': 'system'
        }

        # Extraer métricas genéricas
        metrics = self._extract_generic_metrics(line)
        log_entry.update(metrics)

        return log_entry

    def _extract_weblogic_metrics(self, line):
        """Extraer métricas de WebLogic"""
        metrics = {}
        line_lower = line.lower()

        # Buscar porcentajes (para heap, CPU)
        percent_matches = re.findall(r'(\d+\.?\d*)%', line)
        if percent_matches:
            for i, percent in enumerate(percent_matches[:2]):  # Tomar primeros 2 porcentajes
                try:
                    value = float(percent)
                    if 0 <= value <= 100:
                        if i == 0:
                            metrics['heap_usage_percent'] = value
                        elif i == 1:
                            metrics['cpu_usage_percent'] = value
                except:
                    pass

        # Buscar números relacionados con conexiones
        conn_matches = re.findall(r'connections?.*?(\d+)', line_lower)
        if conn_matches:
            metrics['active_connections'] = int(conn_matches[0])

        # Buscar números relacionados con threads
        thread_matches = re.findall(r'threads?.*?(\d+)', line_lower)
        if thread_matches:
            metrics['active_threads'] = int(thread_matches[0])

        # Buscar tiempos en ms
        time_matches = re.findall(r'(\d+)\s*ms', line_lower)
        if time_matches:
            metrics['response_time_ms'] = float(time_matches[0])

        # Buscar números generales y asignar a métricas si faltan
        if 'heap_usage_percent' not in metrics:
            numbers = re.findall(r'\b(\d+)\b', line)
            if numbers:
                try:
                    metrics['heap_usage_percent'] = float(numbers[0]) % 100
                except:
                    pass

        if 'cpu_usage_percent' not in metrics:
            numbers = re.findall(r'\b(\d+)\b', line)
            if len(numbers) > 1:
                try:
                    metrics['cpu_usage_percent'] = float(numbers[1]) % 100
                except:
                    pass

        return metrics

    def _extract_apache_metrics(self, line):
        """Extraer métricas de access log"""
        metrics = {}

        # Extraer código de estado HTTP
        status_match = re.search(r'"\s+(\d{3})\s+', line)
        if status_match:
            status = int(status_match.group(1))
            metrics['http_status'] = status
            metrics['is_error'] = 1 if status >= 400 else 0

        # Extraer tamaño de respuesta
        size_match = re.search(r'\s+(\d+)\s+$', line)
        if size_match:
            metrics['response_size'] = int(size_match.group(1))

        # Extraer tiempo de respuesta (último número)
        parts = line.split()
        if len(parts) >= 10:
            try:
                metrics['response_time_ms'] = float(parts[-1])
            except:
                pass

        # Generar métricas sintéticas basadas en el contenido
        if 'heap_usage_percent' not in metrics:
            metrics['heap_usage_percent'] = 50 + (hash(line) % 50)  # Valor entre 50-100

        if 'cpu_usage_percent' not in metrics:
            metrics['cpu_usage_percent'] = 30 + (hash(line) % 70)  # Valor entre 30-100

        return metrics

    def _extract_generic_metrics(self, line):
        """Extraer métricas genéricas"""
        metrics = {}

        # Extraer todos los números de la línea
        numbers = re.findall(r'\b(\d+\.?\d*)\b', line)

        if numbers:
            # Asignar números a métricas en orden
            metric_names = [
                'heap_usage_percent',
                'cpu_usage_percent',
                'response_time_ms',
                'active_threads',
                'active_connections'
            ]

            for i, num in enumerate(numbers[:len(metric_names)]):
                try:
                    value = float(num)
                    metric_name = metric_names[i]

                    # Ajustar valores a rangos razonables
                    if 'percent' in metric_name:
                        value = value % 100
                    elif metric_name == 'response_time_ms':
                        if value > 10000:
                            value = value / 100

                    metrics[metric_name] = value
                except:
                    pass

        # Asegurar que tenemos al menos algunas métricas
        if 'heap_usage_percent' not in metrics:
            metrics['heap_usage_percent'] = 50 + (hash(line) % 50)

        if 'cpu_usage_percent' not in metrics:
            metrics['cpu_usage_percent'] = 30 + (hash(line) % 70)

        return metrics

# ============================================================================
# 2. SISTEMA DE ENTRENAMIENTO CON DATOS REALES
# ============================================================================

class RealDataTrainingSystem:
    """Sistema que entrena solo con datos reales de archivos"""

    def __init__(self):
        self.parser = WebLogicLogParser()
        self.training_data = []
        self.model = None
        self.scaler = RobustScaler()
        self.is_trained = False

    def load_training_data(self, filepaths, max_lines_per_file=10000):
        """Cargar datos de entrenamiento desde archivos"""
        print("=" * 60)
        print("CARGANDO DATOS DE ENTRENAMIENTO DESDE ARCHIVOS...")
        print("=" * 60)

        all_logs = []
        total_lines = 0

        for filepath in filepaths:
            if os.path.exists(filepath):
                file_size = os.path.getsize(filepath)
                print(f"  {os.path.basename(filepath)} ({file_size:,} bytes)")

                logs, lines = self.parser.parse_file(filepath, max_lines_per_file)
                all_logs.extend(logs)
                total_lines += lines
            else:
                print(f"  Archivo no encontrado: {filepath}")

        if not all_logs:
            print("ERROR: No se pudieron cargar logs de los archivos")
            return False

        self.training_data = all_logs
        print("=" * 60)
        print(f"TOTAL LOGS CARGADOS: {len(all_logs)}")
        print(f"Lineas procesadas: {total_lines}")
        print(f"Archivos procesados: {len(filepaths)}")

        # Mostrar estadísticas
        self._show_data_statistics()

        return True

    def _show_data_statistics(self):
        """Mostrar estadísticas de los datos cargados"""
        if not self.training_data:
            return

        df = pd.DataFrame(self.training_data)

        print("\nESTADISTICAS DE DATOS:")
        print("-" * 40)
        print(f"Total registros: {len(df)}")

        # Distribución por fuente
        if 'source' in df.columns:
            print("\nDistribucion por fuente:")
            source_counts = df['source'].value_counts()
            for source, count in source_counts.items():
                percentage = (count / len(df)) * 100
                print(f"  {source}: {count} ({percentage:.1f}%)")

        # Métricas disponibles
        metric_columns = []
        for col in df.columns:
            if col not in ['timestamp', 'original_line', 'line_number', 'source', 'log_type']:
                if df[col].notna().any():
                    metric_columns.append(col)

        print(f"\nMetricas disponibles ({len(metric_columns)}):")
        for metric in metric_columns[:10]:  # Mostrar primeras 10
            non_null = df[metric].notna().sum()
            if non_null > 0:
                mean_val = df[metric].mean()
                print(f"  {metric}: {non_null} valores, promedio: {mean_val:.2f}")

    def train_model(self):
        """Entrenar modelo con datos reales"""
        if not self.training_data:
            print("ERROR: No hay datos para entrenar")
            return False

        print("\nENTRENANDO MODELO...")

        # Preparar características para entrenamiento
        X = self._prepare_training_features()

        if X.shape[0] < 100:
            print(f"ERROR: Muy pocas muestras para entrenar. Necesarias: 100, Disponibles: {X.shape[0]}")
            return False

        print(f"  Muestras de entrenamiento: {X.shape[0]}")
        print(f"  Caracteristicas por muestra: {X.shape[1]}")

        # Entrenar Isolation Forest
        self.model = IsolationForest(
            n_estimators=100,
            contamination=0.1,
            random_state=42,
            verbose=0
        )

        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled)
        self.is_trained = True

        print("EXITO: Modelo entrenado exitosamente")

        # Guardar modelo
        self.save_model('modelo_mdw_entrenado.pkl')

        return True

    def _prepare_training_features(self):
        """Preparar características para entrenamiento"""
        df = pd.DataFrame(self.training_data)

        # Seleccionar características clave
        feature_columns = [
            'heap_usage_percent',
            'cpu_usage_percent',
            'response_time_ms',
            'active_threads',
            'active_connections'
        ]

        # Filtrar columnas existentes
        existing_features = [col for col in feature_columns if col in df.columns]

        # Crear matriz de características
        X_list = []

        for _, row in df.iterrows():
            features = []
            valid = True

            for feature in existing_features:
                if feature in row and pd.notna(row[feature]):
                    features.append(float(row[feature]))
                else:
                    # Usar valor por defecto si falta
                    if feature == 'heap_usage_percent':
                        features.append(50.0)
                    elif feature == 'cpu_usage_percent':
                        features.append(50.0)
                    elif feature == 'response_time_ms':
                        features.append(100.0)
                    elif feature == 'active_threads':
                        features.append(10.0)
                    elif feature == 'active_connections':
                        features.append(5.0)

            if len(features) == len(existing_features):
                X_list.append(features)

        return np.array(X_list)

    def predict(self, features_dict):
        """Predecir anomalía para un conjunto de características"""
        if not self.is_trained or self.model is None:
            return {'is_anomaly': False, 'score': 0.0, 'confidence': 0.0}

        # Extraer características en orden
        feature_order = [
            'heap_usage_percent',
            'cpu_usage_percent',
            'response_time_ms',
            'active_threads',
            'active_connections'
        ]

        features = []
        for feature in feature_order:
            if feature in features_dict:
                features.append(float(features_dict[feature]))
            else:
                # Valor por defecto si falta
                features.append(50.0 if 'percent' in feature else 0.0)

        # Escalar y predecir
        try:
            features_scaled = self.scaler.transform([features])
            score = self.model.decision_function(features_scaled)[0]
            is_anomaly = score < -0.1

            return {
                'is_anomaly': bool(is_anomaly),
                'score': float(score),
                'confidence': float(min(1.0, abs(score)))
            }
        except:
            return {'is_anomaly': False, 'score': 0.0, 'confidence': 0.0}

    def save_model(self, filename):
        """Guardar modelo entrenado"""
        if self.is_trained:
            with open(filename, 'wb') as f:
                pickle.dump({
                    'model': self.model,
                    'scaler': self.scaler,
                    'is_trained': True,
                    'training_samples': len(self.training_data) if self.training_data else 0
                }, f)
            print(f"Modelo guardado en {filename}")

    def load_model(self, filename):
        """Cargar modelo previamente entrenado"""
        try:
            with open(filename, 'rb') as f:
                saved = pickle.load(f)

            self.model = saved['model']
            self.scaler = saved['scaler']
            self.is_trained = saved['is_trained']

            print(f"Modelo cargado desde {filename}")
            return True
        except Exception as e:
            print(f"Error al cargar modelo: {e}")
            return False

# ============================================================================
# 3. SISTEMA DE DETECCIÓN EN TIEMPO REAL (CON DATOS REALES)
# ============================================================================

class RealTimeDetectionSystem:
    """Sistema de detección que usa datos reales"""

    def __init__(self, training_system=None):
        self.training_system = training_system or RealDataTrainingSystem()
        self.current_data = []
        self.detection_history = []

    def run_detection_on_loaded_data(self, duration_seconds=30):
        """Ejecutar detección sobre datos ya cargados"""
        if not self.training_system.training_data:
            print("ERROR: No hay datos cargados para detección")
            return

        if not self.training_system.is_trained:
            print("ERROR: Modelo no entrenado")
            return

        print("=" * 60)
        print("EJECUTANDO SISTEMA EN TIEMPO REAL")
        print("=" * 60)

        print(f"Analizando {len(self.training_system.training_data)} registros...")

        stats = {
            'total_processed': 0,
            'anomalies_detected': 0,
            'warnings_generated': 0,
            'critical_alerts': 0
        }

        # Procesar cada registro
        for i, log_entry in enumerate(self.training_system.training_data):
            stats['total_processed'] += 1

            # Extraer características para predicción
            features = {}
            for key in ['heap_usage_percent', 'cpu_usage_percent', 'response_time_ms',
                       'active_threads', 'active_connections']:
                if key in log_entry:
                    features[key] = log_entry[key]

            # Predecir anomalía
            prediction = self.training_system.predict(features)

            # Verificar alertas
            alerts = self._check_alerts(log_entry, prediction)

            # Actualizar estadísticas
            if prediction['is_anomaly']:
                stats['anomalies_detected'] += 1

            if alerts:
                for alert in alerts:
                    if alert.get('severity') == 'CRITICAL':
                        stats['critical_alerts'] += 1
                    else:
                        stats['warnings_generated'] += 1

            # Guardar en historial
            detection_record = {
                'timestamp': datetime.now(),
                'log_entry': log_entry,
                'prediction': prediction,
                'alerts': alerts
            }
            self.detection_history.append(detection_record)

            # Mostrar progreso cada 100 registros
            if (i + 1) % 100 == 0:
                self._print_progress(i + 1, stats, log_entry, prediction, alerts)

        # Mostrar resumen final
        self._print_final_summary(stats)

    def _check_alerts(self, log_entry, prediction):
        """Verificar alertas basadas en métricas y predicciones"""
        alerts = []

        # Alertas por métricas específicas
        metric_thresholds = [
            ('heap_usage_percent', 80, 'WARNING', 90, 'CRITICAL'),
            ('cpu_usage_percent', 75, 'WARNING', 85, 'CRITICAL'),
            ('response_time_ms', 500, 'WARNING', 1000, 'CRITICAL'),
            ('active_threads', 50, 'WARNING', 100, 'CRITICAL')
        ]

        for metric, warn_thresh, warn_sev, crit_thresh, crit_sev in metric_thresholds:
            if metric in log_entry:
                value = log_entry[metric]

                if value >= crit_thresh:
                    alert = {
                        'severity': crit_sev,
                        'metric': metric,
                        'value': value,
                        'threshold': crit_thresh,
                        'message': f"{metric} = {value:.1f} (umbral: {crit_thresh})"
                    }
                    alerts.append(alert)
                elif value >= warn_thresh:
                    alert = {
                        'severity': warn_sev,
                        'metric': metric,
                        'value': value,
                        'threshold': warn_thresh,
                        'message': f"{metric} = {value:.1f} (umbral: {warn_thresh})"
                    }
                    alerts.append(alert)

        # Alerta por anomalía detectada
        if prediction['is_anomaly']:
            alert = {
                'severity': 'CRITICAL',
                'type': 'ANOMALY',
                'score': prediction['score'],
                'message': f"Anomalia detectada (score: {prediction['score']:.3f})"
            }
            alerts.append(alert)

        return alerts

    def _print_progress(self, count, stats, log_entry, prediction, alerts):
        """Imprimir progreso de la detección"""
        print(f"\nRegistro #{count} | Anomalias: {stats['anomalies_detected']} | Alertas: {stats['warnings_generated'] + stats['critical_alerts']}")

        # Mostrar métricas clave
        metrics_display = []
        for metric in ['heap_usage_percent', 'cpu_usage_percent', 'response_time_ms']:
            if metric in log_entry:
                value = log_entry[metric]
                if 'percent' in metric:
                    label = metric.split('_')[0].upper()
                    metrics_display.append(f"{label}: {value:.1f}%")
                elif 'ms' in metric:
                    metrics_display.append(f"Resp: {value:.0f}ms")

        if metrics_display:
            print(f"  {' | '.join(metrics_display)}")

        if prediction['is_anomaly']:
            print(f"  ANOMALIA DETECTADA! Score: {prediction['score']:.3f}")

        if alerts:
            for alert in alerts[:2]:  # Mostrar máximo 2 alertas
                print(f"  {alert['severity']}: {alert['message'][:50]}")

    def _print_final_summary(self, stats):
        """Imprimir resumen final"""
        print("\n" + "=" * 60)
        print("RESUMEN FINAL DE DETECCION")
        print("=" * 60)

        print(f"\nESTADISTICAS:")
        print(f"  Total registros procesados: {stats['total_processed']}")
        print(f"  Anomalias detectadas: {stats['anomalies_detected']}")

        if stats['total_processed'] > 0:
            anomaly_rate = (stats['anomalies_detected'] / stats['total_processed']) * 100
            print(f"  Tasa de anomalias: {anomaly_rate:.2f}%")

        print(f"  Alertas de advertencia: {stats['warnings_generated']}")
        print(f"  Alertas criticas: {stats['critical_alerts']}")

        print(f"\nMODELO:")
        print(f"  Entrenado: {'SI' if self.training_system.is_trained else 'NO'}")
        if self.training_system.training_data:
            print(f"  Datos de entrenamiento: {len(self.training_system.training_data)}")

        # Mostrar últimas alertas críticas
        critical_alerts = []
        for record in self.detection_history[-20:]:  # Últimos 20 registros
            if record['alerts']:
                for alert in record['alerts']:
                    if alert.get('severity') == 'CRITICAL':
                        critical_alerts.append(alert)

        if critical_alerts:
            print(f"\nULTIMAS ALERTAS CRITICAS (max 5):")
            for i, alert in enumerate(critical_alerts[-5:], 1):
                print(f"  {i}. {alert['message'][:60]}")

        print(f"\nDETECCION COMPLETADA EXITOSAMENTE!")

# ============================================================================
# 4. FUNCIÓN PRINCIPAL DE EJECUCIÓN
# ============================================================================

def ejecutar_sistema_con_archivos_reales():
    """Función principal para ejecutar el sistema con archivos reales"""
    print("=" * 60)
    print("SISTEMA DE DETECCION CON ARCHIVOS REALES")
    print("=" * 60)

    # Archivos específicos del usuario
    USER_FILES = [
        '/content/archivosaccess.txt',
        '/content/archivosout.out',
        '/content/archivoslog.log0'
    ]

    # Verificar archivos
    print("\nVERIFICANDO ARCHIVOS:")
    print("-" * 50)

    existing_files = []
    for filepath in USER_FILES:
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath)
            filename = os.path.basename(filepath)
            print(f"  OK: {filename} ({file_size:,} bytes)")
            existing_files.append(filepath)
        else:
            filename = os.path.basename(filepath)
            print(f"  NO ENCONTRADO: {filename}")

    if not existing_files:
        print("\nERROR: No se encontraron archivos.")
        print("Por favor, sube los archivos primero.")
        return None

    print(f"\nArchivos validos encontrados: {len(existing_files)}")

    # Preguntar opciones de procesamiento
    print("\n" + "=" * 60)
    print("OPCIONES DE PROCESAMIENTO:")
    print("1. Usar todos los archivos encontrados")
    print("2. Seleccionar archivos especificos")
    print("3. Probar con un archivo de ejemplo generado")

    try:
        option = input("\nSeleccione opcion (1-3): ").strip()
    except:
        option = "1"

    if option == "1":
        files_to_process = existing_files
    elif option == "2":
        print("\nArchivos disponibles:")
        for i, filepath in enumerate(existing_files, 1):
            filename = os.path.basename(filepath)
            print(f"  {i}. {filename}")

        try:
            selections = input("\nSeleccione numeros (ej: 1,2,3): ").strip()
            selected_indices = [int(x.strip()) - 1 for x in selections.split(',') if x.strip().isdigit()]
            files_to_process = [existing_files[i] for i in selected_indices if 0 <= i < len(existing_files)]
        except:
            files_to_process = existing_files
    elif option == "3":
        # Crear archivo de ejemplo
        print("\nCreando archivo de ejemplo...")
        files_to_process = [create_sample_file()]
    else:
        files_to_process = existing_files

    # Crear sistema de entrenamiento
    training_system = RealDataTrainingSystem()

    try:
        # 1. Cargar datos de entrenamiento
        success = training_system.load_training_data(
            files_to_process,
            max_lines_per_file=10000
        )

        if not success:
            print("\nNo se pudieron cargar datos suficientes.")
            return training_system

        # 2. Entrenar modelo
        trained = training_system.train_model()

        if not trained:
            print("\nNo se pudo entrenar el modelo.")

            # Intentar cargar modelo existente
            if os.path.exists('modelo_mdw_entrenado.pkl'):
                print("Intentando cargar modelo existente...")
                training_system.load_model('modelo_mdw_entrenado.pkl')
            else:
                print("No hay modelo existente disponible.")
                return training_system

        # 3. Ejecutar detección
        detection_system = RealTimeDetectionSystem(training_system)
        detection_system.run_detection_on_loaded_data(duration_seconds=30)

        return detection_system

    except Exception as e:
        print(f"\nERROR durante la ejecucion: {e}")
        import traceback
        traceback.print_exc()

        return training_system

def create_sample_file():
    """Crear archivo de ejemplo para pruebas"""
    sample_content = """####<Oct 16, 2025 11:02:51 PM CST> <Info> <JDBC> <Server1> <JDBC Connection Pool Usage: 65%>
####<Oct 16, 2025 11:02:52 PM CST> <Warning> <JDBC> <Server1> <High CPU Usage: 85%>
####<Oct 16, 2025 11:02:53 PM CST> <Error> <JDBC> <Server1> <Response Time: 1200ms>
####<Oct 16, 2025 11:02:54 PM CST> <Info> <JDBC> <Server1> <Active Threads: 45>
####<Oct 16, 2025 11:02:55 PM CST> <Warning> <JDBC> <Server1> <Heap Usage: 88%>
192.168.1.1 - - [16/Oct/2025:11:03:00 -0500] "POST /endpoint" 200 1024 150.5
192.168.1.2 - - [16/Oct/2025:11:03:01 -0500] "GET /status" 500 512 2000.0
<INFO> <Server> <Memory Configuration: Xms=1024m, Xmx=2048m>
<WARNING> <Server> <Active Connections: 95>
"""

    sample_path = '/content/archivo_ejemplo.log'
    with open(sample_path, 'w') as f:
        f.write(sample_content)

    print(f"Archivo de ejemplo creado: {sample_path}")
    return sample_path

# ============================================================================
# 5. EJECUCIÓN DIRECTA EN COLAB
# ============================================================================

if __name__ == "__main__":
    # Ejecutar directamente el sistema con archivos reales
    sistema = ejecutar_sistema_con_archivos_reales()

    if sistema is None:
        print("\nNo se pudo ejecutar el sistema.")
        print("\nINSTRUCCIONES PARA SUBIR ARCHIVOS:")
        print("1. Ejecuta en una celda separada:")
        print("   from google.colab import files")
        print("   uploaded = files.upload()")
        print("2. Selecciona tus archivos: archivosaccess.txt, archivosout.out, archivoslog.log0")
        print("3. Vuelve a ejecutar esta celda")

SISTEMA DE DETECCION CON ARCHIVOS REALES

VERIFICANDO ARCHIVOS:
--------------------------------------------------
  OK: archivosaccess.txt (1,765,270 bytes)
  OK: archivosout.out (5,354,962 bytes)
  OK: archivoslog.log0 (5,120,005 bytes)

Archivos validos encontrados: 3

OPCIONES DE PROCESAMIENTO:
1. Usar todos los archivos encontrados
2. Seleccionar archivos especificos
3. Probar con un archivo de ejemplo generado

Seleccione opcion (1-3): 1
CARGANDO DATOS DE ENTRENAMIENTO DESDE ARCHIVOS...
  archivosaccess.txt (1,765,270 bytes)
Leyendo archivo: archivosaccess.txt
  Tamano: 1,765,270 bytes
  Formato detectado: apache
  Procesadas 1000 líneas...
  Procesadas 2000 líneas...
  Procesadas 3000 líneas...
  Procesadas 4000 líneas...
  Procesadas 5000 líneas...
  Procesadas 6000 líneas...
  Procesadas 7000 líneas...
  Procesadas 8000 líneas...
  Procesadas 9000 líneas...
  Procesadas 10000 líneas...
  Total de lineas: 10000
  Logs parseados: 10000
  archivosout.out (5,354,962 bytes)
Leyendo